In [5]:
from time import sleep
from tqdm import trange
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from urllib import request
from konlpy.tag import Kkma
import pandas as pd

kma = Kkma()

## 01. ChromeDriver 설정
driver = webdriver.Chrome()


## 02. 에브리타임 접속 (로그인 -> 동아리게시판 -> 교내)
driver.get('https://everytime.kr/login')
sleep(3)

id_every = driver.find_element(by=By.NAME, value="userid")
id_every.send_keys("dianecy1121")
sleep(1)

pwd_every = driver.find_element(by=By.NAME, value="password")
pwd_every.send_keys("vishnu#kid020612")

login_btn = driver.find_element(by=By.TAG_NAME, value="input")
login_btn.send_keys(Keys.RETURN)
sleep(2)

driver.find_element(by=By.XPATH, value ='/html/body/div[1]/div/div[3]/ul/li/a').click()
sleep(1)

driver.find_element(by=By.CSS_SELECTOR, value = '#container > div.wrap.categories > div:nth-child(2) > span').click()
sleep(1)


## 03. url 추출 함수 메서드 정의
words = []
s_dict ={'dname' : [], 'durl': [], 'dcontent':[]}
def nextpage() :
    global words, s_dict
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    souplist = soup.find('div', class_ = 'wrap articles')
    dataname = souplist.select("h2")
    dataurl = souplist.select("article a.article")
    datatext = souplist.select("p")
    
    for dname, durl in zip(dataname, dataurl) :
        url = r'https://everytime.kr'+durl.get('href')
        s_dict['dname'].append(dname.get_text())
        s_dict['durl'].append(url)
    
    for datas in datatext :
        page_tex = datas.get_text()
        words += kma.nouns(page_tex)

## 04. 페이지에서 텍스트를 추출
for pageloop in range(15) :
    nextpage()
    if pageloop == 0 :
        driver.find_element(by=By.XPATH, value = '/html/body/div[2]/div[3]/div[2]/a').click()
        sleep(1.5)
    elif pageloop == 1:
        driver.find_element(by=By.XPATH, value = '/html/body/div[2]/div[3]/div[2]/a[2]').click()
        sleep(1.5)
    else :
        driver.find_element(by=By.XPATH, value ='/html/body/div[2]/div[3]/div[2]/a[3]').click()
        sleep(1.5)

In [6]:
for idx in trange(len(s_dict['durl'])) :
    driver.get(s_dict['durl'][idx])
    sleep(3.5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    s_dict['dcontent'].append(soup.find_all('article')[0].find('p').text)
    sleep(0.2)

df = pd.DataFrame(s_dict)
df

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [19:21<00:00,  3.87s/it]


,dname,durl,dcontent
0,관악 K-POP 보컬·댄스동아리 1기 추가모집,https://everytime.kr/418781/v/257936908,안녕하세요!관악 K-POP 보컬·댄스 동아리에서 함께 하실 분들을 모집합니다!케이팝...
1,게임/앱 개발 동아리 Happetite에서 신입 PO를 모집합니다! 🔥,https://everytime.kr/418781/v/257918632,게임/앱 개발 동아리 Happetite에서 신입 PO를 모집합니다! 🔥Happeti...
2,D-7 서울대 보컬힙합동아리 츄플 리크루팅!!!,https://everytime.kr/418781/v/257913592,‼️🥳~츞하~🥳‼️여러부우운😍😍 모두 안녕하세요😘서울대학교의❗️One❗️And❗️O...
3,[D-3][2022 피움 여름 정원 프로젝트],https://everytime.kr/418781/v/257875016,여름방학 동안 220동 정원을 함께 가꿀 피움 회원을 모집합니다! 8주간의 프로젝트...
4,🌼수도권 연합 보드게임동아리 오해(오늘 뭐해?)에서 4기 부원들을 모집합니다🌼,https://everytime.kr/418781/v/257837447,오늘 뭐해❓우리랑 놀자🧡 보드게임동아리 오해(오늘 뭐해?)에서 신입부원을 모집합니다...
...,...,...,...
295,❗️D-2❗️👏서울대학교 관악사 토론스피치 동아리 ‘아크로폴리스’에서 신입부원을 모...,https://everytime.kr/418781/v/239704167,👏👏서울대학교 관악사 토론스피치 동아리 ‘아크로폴리스’에서 신입부원을 모집합니다👏👏...
296,🔥서울대학교 보컬&힙합 동아리 Triple-H 믹스테잎 홍보입니다🎤,https://everytime.kr/418781/v/239701969,🎼츞~하 ❗️‼️서울대학교 보컬&힙합 동아리 Triple-H의 겨울 녹음 프로젝트의...
297,😎코딩 몰라도 개발할 수 있다? DWNC 1기 모집,https://everytime.kr/418781/v/239675413,"안녕하세요.노코드(no-code)로 아이디어 검증하기, DWNC 동아리 1기를 모집..."
298,♥️[D-DAY] 놈아즈 3기 모집♥️,https://everytime.kr/418781/v/239672899,🔥🔥[D-DAY][D-DAY][D-DAY]🔥🔥😆NOMADS 3기 모집(3/5-3/1...


In [ ]:
driver.get('https://everytime.kr/418781/v/237547768')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
sleep(1)
img = soup.find_all('article')[0].find_all('img')
img[1].get('src')

##### 이미지가 멀티로 묶여있으면(아래 url) 어쩔건데..???????
driver.get('https://everytime.kr/418781/v/252680628')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
sleep(1)
img = soup.find_all('article')[0].find_all('img')

In [ ]:
from collections import Counter

count = Counter(words)
stop_list = ["2022년도", "예정", "최강", "부탁드립", "시간", "시작", "톨릭", "지속", "전공", "년도", "올해", "푸른", "서포터", "나무재단", "체결", "탄생", "경영대", "학부생도", "생도", "대학원생도", "페이스", "마지막", "기초", "사회대", "영문", "영문극회", "25", "리어", "창설", "아트", "단대불문", "대불",  "푸른나무", "총장", "자격", "환영", "카펠라", "인스트", "카펠라동아리", "겨울", "32", "언론인", "11일","씽킹", "오늘", "방안", "16", "로드", "인원", "공지", "준비", "내일", "자랑", "문의", "동아", "드림컨설턴트", "컨설턴트", "11", "오후", "연락", "업로드", "분야", "관련", "이상", "기타", "31", "지금", "지원자", "누구" ,"사랑", "지원서", "모집기간", "역사", "14기", "링크", "당신", "모집대상", "대표", "관악", "학과", "중앙동아리", "불문", "학회원", "문제", "소리", "사람", "학부", "하나", "단체", "진행","부탁", "59", "자정", "7기", "상시모집", "아래", "현재", "3기", "회원분", "스타트", "사회적", "측정",  "마작", "연장", "사범대", "봄학기", "성장", "신청", "최고", "드림", "서울시", "소식", "나무", "연극동아리", "22","서울", "무이", "1기", "다양", "12", "14", "13", "모임", "공식", "안내", "신입회원", "신입단원", "기수", "학번", "가능", "이번", "18", "중앙", "대상", "23", "24", "19", "20", "21", "3월", "연합", "상시", "단원", "2기", "유일무이", "2022년", "모집", "학교", "안녕", "유일", "활동", "지원", "대학", "관심", "저희", "추가", "소속", "기간", "마감", "실력", "학생", "소개", "우리", "멤버", "부원", "회원", "학기", "하세", "여러분","대학교", "총장배", "마무리", "신입부원", "리크루팅", "추가모집", "안녕하세요", "1학기", "2학기", "안녕하세","서울대학교", "서울대", "18기", "19기", "20기","21기", "22기", "23기", "24기", "2022", "년", "19", "동아리", "신입", "신입기수", "페이스북"]
tag_count = []
tags = []

for n,c in count.most_common(8000) :
    dics = {'tag':n, 'count':c}
    if dics['tag'] not in stop_list and len(dics['tag']) >=2 and len(tags)<=85:
        tag_count.append(dics)
        tags.append(dics['tag'])

for tag in tag_count :
    print(tag['tag'].strip(), end='\t\t')
    print("{}".format(tag['count']))